In [1]:
import pandas as pd
import os


In [2]:
temp = pd.read_csv(
    "D:\Projects\Hackathons\shell hacks\mitre\FIU_Phishing_Mitre_Dataset.csv", 
    na_values=['NA', '?'])
temp.dropna()

df =pd.DataFrame(columns = ['create_age(months)','expiry_age(months)','update_age(days)','lengthOfUrl','#ofDots','#of/','#of-','firstURL'])
df['create_age(months)'] = temp['create_age(months)']
df['expiry_age(months)'] = temp['expiry_age(months)']
df['update_age(days)'] = temp['update_age(days)']
df['lengthOfUrl'] = temp['URL'].apply(len)
df['#ofDots'] = temp['URL'].apply(lambda x: x.count('.'))
df['#of/'] = temp['URL'].apply(lambda x: x.count('/'))
df['#of-'] = temp['URL'].apply(lambda x: x.count('-'))
df['Label'] = temp['Label']

start = 'http://'
end = '/'
df['firstURL'] = temp['URL'].apply(lambda x: x[x.find(start)+len(start):x.rfind(end)])
df['firstURL'] = df['firstURL'].apply(len)

export_csv = df.to_csv(r'TrainingSet.csv',index=None,header=True)


In [7]:
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils import shuffle
from tensorflow.keras.regularizers import l2

In [30]:
df_new = pd.read_csv(
    "D:/Projects/Hackathons/shell hacks/mitre/traing_data.csv",
    na_values=['NA','?'])

#df_new[df_new < 0] = 0
#df_new['create_age(months)'] = zscore(['create_age(months)'])
#df_new['expiry_age(months)'] = zscore(['expiry_age(months)'])
#df_new['update_age(days)'] = zscore(['update_age(days)'])
df_new = shuffle(df_new)


x_columns = df_new.columns.drop('Label')
x = df_new[x_columns].values
dummies = pd.get_dummies(df_new['Label']) # Classification
species = dummies.columns
y = dummies.values


x_train, x_test, y_train, y_test = train_test_split(    
    x, y, test_size=0.20, random_state=42)
df_new

,create_age(months),expiry_age(months),update_age(days),lengthOfUrl,#ofDots,#of/,#of-,firstURL,Label
4346,193,11,55,27,4,3,0,16,0
4192,5,7,150,53,5,4,0,32,1
4439,222,6,178,32,5,3,0,21,0
4317,-1,-1,-1,73,4,5,2,38,1
2486,13,11,53,27,6,3,0,16,0
...,...,...,...,...,...,...,...,...,...
1222,71,1,316,73,6,6,0,28,1
126,41,7,520,41,6,3,0,31,0
4538,-1,-1,-1,59,6,7,0,36,1
134,-1,2,409,27,6,3,0,16,0


In [31]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(100, input_dim = x.shape[1], activation = 'relu',kernel_regularizer = l2(0.01)),
    tf.keras.layers.Dense(50, activation = 'relu',kernel_regularizer = l2(0.01)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(25, activation = 'relu',kernel_regularizer = l2(0.01)),
    tf.keras.layers.Dense(y.shape[1], activation='softmax')
])

In [32]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto', restore_best_weights=True)

In [ ]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor],verbose=2,epochs=1000)


Train on 3839 samples, validate on 960 samples
Epoch 1/1000
3839/3839 - 1s - loss: 4.2868 - accuracy: 0.6765 - val_loss: 1.4773 - val_accuracy: 0.8396
Epoch 2/1000
3839/3839 - 1s - loss: 1.7601 - accuracy: 0.7775 - val_loss: 1.0859 - val_accuracy: 0.8750
Epoch 3/1000
3839/3839 - 1s - loss: 1.4082 - accuracy: 0.8036 - val_loss: 1.0520 - val_accuracy: 0.8583
Epoch 4/1000


In [29]:
from sklearn.metrics import accuracy_score
import numpy as np

pred = model.predict(x_test)
predict_classes = np.argmax(pred,axis=1)
expected_classes = np.argmax(y_test,axis=1)
correct = accuracy_score(expected_classes,predict_classes)
print(f"Accuracy: {correct}")

Accuracy: 0.8760416666666667
